Convert to ECOSTRESS Spectral Library   
Susan Meerdink   
12/8/2016   
https://github.com/susanmeerdink/ASTER-Spectral-Library    
This code converts a csv of spectra and metadata into ECOSTRESS spectral library format. This project contains example spectra and metadata in the repo.    
This code takes four inputs:  
1. Metadata for spectra csv - no commas in metadata fields
2. ASD Spectra csv - must have the same order as metadata
3. Nicolet Spectra csv - must have the same order as metadata
4. Output directory for ECOSTRESS spectral library files

In [1]:
# Import Functions
import numpy as np

In [2]:
def read_file(filename):
    """
    Function for reading in metadata, ASD, or Nicolet spectra from csv.
    
    Parameters:
    -----------
    1) filename: The filename of the csv containing asd spectra information.
    
    Returns:
    --------
    1) array: numpy array that contains the parsed file info
    2) header: string array containing the names
    """
    inFile = open(filename, 'r')  # Open ASD spectra file
    numRow = 0  # Keeps a counter of the number of rows
    array = []  # empty array to hold ASD spectra data
    
    # Loop through file
    for line in inFile:
        strLine = line.split(",")

        if numRow == 0:  # if this is the first row, then grab column names
            strLine[len(strLine)-1] = strLine[len(strLine) - 1].rstrip('\n')  # removing newline character from last field
            header = strLine  # Store the first line for output files
            numRow += 1  # advance counter
        else:  # if it isn't the first row
            array.append(strLine)
            
    return array, header

In [3]:
def write_file(outDir, meta, asd, nicolet, headersMeta, headersASD, headersNicolet):
    """
    Function for writing an ECOSTRESS Spectral Library file.
    
    Parameters:
    -----------
    1) meta: a numpy array containing strings associated with the metadata fields for one sample
    2) asd: a numpy array containing asd spectra for one sample
    3) nicolet: a numpy array containing nicolet spectra for one sample
    
    Output File:
    --------
    1) Outputs a text file containing the metadata, ASD, and Nicolet spectra for one sample.
    See readme for formatting information.
    """
    # Output file name format: location.instrument.type.class.genus.species.samplenumber.filetype.txt
    # Example file name format: jpl.asdnicolet.npv.bark.abies.concolor.vh311.spectrum.txt
    if 'non' in meta[1]:
        outFileName = outDir + (meta[6] + '.asdnicolet.' + 'nonphotosyntheticvegetation' + '.' + 
                            meta[2] + '.' + meta[3] + '.' + meta[4] + '.' + 
                            meta[5] + '.spectrum.txt').lower()
    else:
        outFileName = outDir + (meta[6] + '.asdnicolet.' + meta[1] + '.' + 
                            meta[2] + '.' + meta[3] + '.' + meta[4] + '.' + 
                            meta[5] + '.spectrum.txt').lower()
    addinfoLine = (meta[6] + '.asdnicolet.' + meta[1] + '.' + meta[2] + '.' + meta[3] + 
                   '.' + meta[4] + '.' + meta[5] + '.ancillary.txt').lower()
    outFile = open(outFileName, 'w')  # open file
    
    # Output Metadata
    for i in range(len(meta)):  # loop through metadata fields
        if i == len(headersMeta) - 1:
            if 'TRUE' in meta[i]:  # If the last column of metadata csv is set to True there will be an ancillary file associated
                outFile.write(headersMeta[i] + ': ' + addinfoLine + '\n')
                outFile.write('\n')
            else:
                outFile.write(headersMeta[i] + ': \n')
                outFile.write('\n')
        else:
            outFile.write(headersMeta[i] + ': ' + meta[i] + '\n')

    # Output ASD data
    for j in range(1, len(headersASD)):  # loop through asd fields
        # Leading space so that wavelengths are right justified 
        outFile.write(" " + headersASD[j] + "\t" + ("{0:.6s}".format(asd[j])) + '\n')

    # Output Nicolet data
    for k in range(1, len(headersNicolet)):  # loop through nicolet fields
        if nicolet[k] != "":
            if len(headersNicolet[k]) < 6: # Add leading space so that wavelengths are right justified 
                outFile.write(" " + headersNicolet[k] + "\t" + ("{0:.6s}".format(nicolet[k])) + '\n')
            else:  # Do not add leading space because there are 6 characters
                outFile.write(headersNicolet[k] + "\t" + ("{0:.6s}".format(nicolet[k])) + '\n')

    outFile.close()  # close file so it can be reused at the beginning

In [4]:
def create_spectral_library_files(metaFilename, asdFilename, nicoletFilename, outDir):
    """
    Function for reading in metadata, ASD, or Nicolet spectra from csv.
    
    Parameters:
    -----------
    1) metaFilename: file path for csv containing metadata
    2) asdFilename: file path for csv containing asd spectra
    3) nicoletFilename: file path for csv containing nicolet spectra
    4) outDir: file path for the output location of spectra library files
    
    Returns:
    --------
    """
    # Read in files and grab data/headers
    arrayMeta, headersMeta = read_file(metaFilename)
    arrayASD, headersASD = read_file(asdFilename)
    arrayNicolet, headersNicolet = read_file(nicoletFilename)
    
    # Loop through each row and create output file
    for row in range(len(arrayMeta)):
        meta = arrayMeta[row]
        asd = arrayASD[row]
        nicolet = arrayNicolet[row]
        write_file(outDir, meta, asd, nicolet, headersMeta, headersASD, headersNicolet)
    
    print 'Finished converting %s samples to ECOSTRESS spectral library files' % len(arrayMeta)

In [5]:
# Example Files
directory = "C:\\Users\\Susan\\Documents\\GitHub\\ASTER-Spectral-Library\\"
inMetaFileName = directory + "Example_Metadata.csv"
inASDFileName = directory + "Example_Spectra_ASD.csv"
inNicoletFileName = directory + "Example_Spectra_Nicolet.csv"
outDir = directory + "Output Spectral Libraries\\"

create_spectral_library_files(inMetaFileName, inASDFileName, inNicoletFileName, outDir)

Finished converting 14 samples to ECOSTRESS spectral library files


In [6]:
# Huntington Gardens Spectra
directory = "F:\\Dropbox\\Analysis\\ECOSTRESS Spectral Library\\Input Spectral Library Files\\"
inMetaFileName = directory + "Huntington_Gardens_Metadata.csv"
inASDFileName = directory + "Huntington_Gardens_Spectra_ASD.csv"
inNicoletFileName = directory + "Huntington_Gardens_Spectra_Nicolet.csv"
outDir = "F:\\Dropbox\\Analysis\\ECOSTRESS Spectral Library\\ECOSTRESS Spectral Library Files\\"

create_spectral_library_files(inMetaFileName, inASDFileName, inNicoletFileName, outDir)

Finished converting 206 samples to ECOSTRESS spectral library files


In [7]:
# HyspIRI Spectra
directory = "F:\\Dropbox\\Analysis\\ECOSTRESS Spectral Library\\Input Spectral Library Files\\"
inMetaFileName = directory + "HyspIRI_Metadata.csv"
inASDFileName = directory + "HyspIRI_Spectra_ASD.csv"
inNicoletFileName = directory + "HyspIRI_Spectra_Nicolet.csv"
outDir = "F:\\Dropbox\\Analysis\\ECOSTRESS Spectral Library\\ECOSTRESS Spectral Library Files\\"

create_spectral_library_files(inMetaFileName, inASDFileName, inNicoletFileName, outDir)

Finished converting 292 samples to ECOSTRESS spectral library files


In [8]:
# Harvard Spectra
directory = "F:\\Dropbox\\Analysis\\ECOSTRESS Spectral Library\\Input Spectral Library Files\\"
inMetaFileName = directory + "Harvard_Metadata.csv"
inASDFileName = directory + "Harvard_Spectra_ASD.csv"
inNicoletFileName = directory + "Harvard_Spectra_Nicolet.csv"
outDir = "F:\\Dropbox\\Analysis\\ECOSTRESS Spectral Library\\ECOSTRESS Spectral Library Files\\"

create_spectral_library_files(inMetaFileName, inASDFileName, inNicoletFileName, outDir)

Finished converting 47 samples to ECOSTRESS spectral library files


In [9]:
# NPV and Miscellanous Spectra
directory = "F:\\Dropbox\\Analysis\\ECOSTRESS Spectral Library\\Input Spectral Library Files\\"
inMetaFileName = directory + "NPV_Misc_Metadata.csv"
inASDFileName = directory + "NPV_Misc_Spectra_ASD.csv"
inNicoletFileName = directory + "NPV_Misc_Spectra_Nicolet.csv"
outDir = "F:\\Dropbox\\Analysis\\ECOSTRESS Spectral Library\\ECOSTRESS Spectral Library Files\\"

create_spectral_library_files(inMetaFileName, inASDFileName, inNicoletFileName, outDir)

Finished converting 39 samples to ECOSTRESS spectral library files


In [10]:
# Miscellanous Vegetation Spectra
directory = "F:\\Dropbox\\Analysis\\ECOSTRESS Spectral Library\\Input Spectral Library Files\\"
inMetaFileName = directory + "Veg_Misc_Metadata.csv"
inASDFileName = directory + "Veg_Misc_Spectra_ASD.csv"
inNicoletFileName = directory + "Veg_Misc_Spectra_Nicolet.csv"
outDir = "F:\\Dropbox\\Analysis\\ECOSTRESS Spectral Library\\ECOSTRESS Spectral Library Files\\"

create_spectral_library_files(inMetaFileName, inASDFileName, inNicoletFileName, outDir)

Finished converting 8 samples to ECOSTRESS spectral library files
